In [4]:
import os
from dotenv import load_dotenv
import requests
import json
from IPython.display import display, HTML, Markdown

In [6]:
load_dotenv()
KEY = os.getenv("GOOGLE_BOOKS_API_KEY")
BASE_URL = "https://www.googleapis.com/books/v1/volumes"

# Define the genres and other parameters
GENRES = ['mystery', 'fantasy', 'science fiction', 'romance', 'historical fiction', 'true crime']
RESULTS_PER_PAGE = 40  # Maximum allowed per request
MAX_BOOKS = 700        # Maximum books to fetch per genre

def fetch_books_for_genre(genre, max_books=500):
    """Fetches up to `max_books` newest books for a given genre."""
    books = []
    start_index = 0
    while start_index < max_books:
        params = {
            'q': f"subject:{genre}",
            'orderBy': 'newest',
            'startIndex': start_index,
            'maxResults': RESULTS_PER_PAGE,
            'key': KEY
        }
        response = requests.get(BASE_URL, params=params)
        if response.status_code != 200:
            print(f"Error fetching data for {genre} at index {start_index}: {response.status_code}")
            break
        data = response.json()
        # If no 'items' in the response, then no more results are available.
        if 'items' not in data:
            break
        items = data['items']
        books.extend(items)
        # If fewer results than requested are returned, we've reached the end.
        if len(items) < RESULTS_PER_PAGE:
            break
        start_index += RESULTS_PER_PAGE
    return books[:max_books]

def main():
    all_genre_books = {}
    for genre in GENRES:
        print(f"Fetching books for genre: {genre}")
        books = fetch_books_for_genre(genre, MAX_BOOKS)
        print(f"Fetched {len(books)} books for genre: {genre}")
        all_genre_books[genre] = books

    # Optionally, save the results to a JSON file
    with open('books_by_genre.json', 'w', encoding='utf-8') as f:
        json.dump(all_genre_books, f, ensure_ascii=False, indent=4)
    
    # Print summary counts per genre
    for genre, books in all_genre_books.items():
        print(f"{genre}: {len(books)} books fetched")

if __name__ == '__main__':
    main()

Fetching books for genre: mystery
Fetched 200 books for genre: mystery
Fetching books for genre: fantasy
Fetched 160 books for genre: fantasy
Fetching books for genre: science fiction
Fetched 320 books for genre: science fiction
Fetching books for genre: romance
Fetched 200 books for genre: romance
Fetching books for genre: historical fiction
Fetched 160 books for genre: historical fiction
Fetching books for genre: true crime
Fetched 1 books for genre: true crime
mystery: 200 books fetched
fantasy: 160 books fetched
science fiction: 320 books fetched
romance: 200 books fetched
historical fiction: 160 books fetched
true crime: 1 books fetched


In [7]:
with open('books_by_genre.json', 'r', encoding='utf-8') as f:
    all_genre_books = json.load(f)

save_books_to_csv(all_genre_books, filename='Googlebooks.csv')


Books data saved to books.csv
